In [ ]:
def xgboost_opt(features, booster, n_trials):
    X, y = train_df[features], train_df.Class
    
    def objective(trial):
        bll_list = list()
        
        params = {
            "n_estimators": 3000, # trial.suggest_int('n_estimators', 100, 1000, step=100),
            "random_state": 14062023,
            "early_stopping_rounds": 100,
            "verbosity": 0,
            # disbalance ratio between 0 and 1 classes
            "scale_pos_weight": 4.71,
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            # use exact for small dataset.
            "tree_method": "exact",
            # defines booster, gblinear for linear functions.
            "booster": booster, # trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]), 
            # L1 regularization weight.
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            # L2 regularization weight.
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            # sampling ratio for training data.
            "subsample": trial.suggest_float("subsample", 0.4, 1.0),
            # sampling according to each tree.
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        }

        if params["booster"] in ["gbtree", "dart"]:
            params["learning_rate"] = trial.suggest_float("learning_rate", 1e-3, 0.1, log=True) # alias eta
            # maximum depth of the tree, signifies complexity of the tree.
            params["max_depth"] = trial.suggest_int("max_depth", 3, 10)
            # minimum child weight, larger the term more conservative the tree.
            params["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
            # defines how selective algorithm is.
            params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

        if params["booster"] == "dart":
            params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
            params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
        
        for i in range(CFG.n_optimize_repeats):
            print(f'{blu}Repeat #{i+1}')

            kf = MultilabelStratifiedKFold(n_splits=CFG.n_optimize_folds, shuffle=True, random_state=8062023+i)

            # Stratify based on Class and Alpha (3 types of conditions)
            for fold, (train_idx, val_idx) in enumerate(kf.split(X=train_df[features], y=greeks.iloc[:,1:3]), start = 1): 

                # Split the dataset according to the fold indexes.
                X_train = X.iloc[train_idx]
                X_val = X.iloc[val_idx]
                y_train = y.iloc[train_idx]
                y_val = y.iloc[val_idx]

                # Learning
                model = xgb.XGBClassifier(**params)
#                 bst = xgb.train(param, dtrain, num_boost_round=10000,
#                                 evals=[(dtrain, 'train'), (dvalid, 'valid')], 
#                                 verbose_eval=1000, early_stopping_rounds=100) 
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=1000)
                # Predict
#                 preds = bst.predict(dvalid)
                preds = model.predict_proba(X_val)[:,1]
                # Evaluation
                bll = balanced_log_loss(y_val, preds)
                bll_list.append(bll)
                
        return np.mean(bll_list)
            
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    df = study.trials_dataframe()
    df.sort_values('value').iloc[:, [1] + list(range(5, 14))]
    df.to_csv(f'optuna_xgb.csv')
            
if CFG.xgb_optimize:
    for booster in ["gbtree", "gblinear"]: # , "dart"]:
        xgboost_opt(features, booster, n_trials=1000)